In [ ]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random

In [ ]:
nltk.download('twitter_samples')

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [ ]:
print(len(all_positive_tweets))
print(len(all_negative_tweets))


5000
5000


In [ ]:
# Our selected sample. Complex enough to exemplify each step
tweet = all_positive_tweets[2277]
print(tweet)

My beautiful sunflowers on a sunny Friday morning off :) #sunflowers #favourites #happy #Friday off… https://t.co/3tfYom0N1i


In [ ]:
# download the stopwords from NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import string
from nltk.stem import PorterStemmer
ps=PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
stopwords=nltk.corpus.stopwords.words('english')
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
def clean_text(text):
  stopwords=nltk.corpus.stopwords.words('english')
  ps = PorterStemmer()
  text=re.sub(r'[^A-Z a-z]','',text)
  text=re.sub(r'^# @ _ ',' ',text)
  text=text.lower()
  text=text.split()
  text=[ps.stem(word) for word in text if word not in stopwords]
  text=" ".join(text)
  return text


In [ ]:
all_positive_tweets[:5]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days']

In [ ]:
clean_text(all_positive_tweets[0])

'followfriday franceint pkuchli milipolpari top engag member commun week'

In [ ]:
for i in range(len(all_positive_tweets)):
  all_positive_tweets[i]=clean_text(all_positive_tweets[i])

In [ ]:
for i in range(len(all_negative_tweets)):
  all_negative_tweets[i]=clean_text(all_negative_tweets[i])

10000

In [ ]:
tweeTtok=all_positive_tweets+all_negative_tweets

tweeTtok[:5]

['followfriday franceint pkuchli milipolpari top engag member commun week',
 'lambja hey jame odd pleas call contact centr abl assist mani thank',
 'despiteoffici listen last night bleed amaz track scotland',
 'side congrat',
 'yeaaaah yippppi accnt verifi rqst succeed got blue tick mark fb profil day']

## Tokenize the string

In [ ]:
from nltk.tokenize import word_tokenize
def process_tweet(tweets):
  tokenized_tweets = []  # Initialize an empty list to store tokenized tweets
  for tweet in tweets:
      tokenized_words = word_tokenize(tweet)
      tokenized_tweets.append(tokenized_words)
  return tokenized_tweets

In [ ]:
tweeTtok=process_tweet(tweet)
tweeTtok

[['followfriday',
  'franceint',
  'pkuchli',
  'milipolpari',
  'top',
  'engag',
  'member',
  'commun',
  'week'],
 ['lambja',
  'hey',
  'jame',
  'odd',
  'pleas',
  'call',
  'contact',
  'centr',
  'abl',
  'assist',
  'mani',
  'thank'],
 ['despiteoffici',
  'listen',
  'last',
  'night',
  'bleed',
  'amaz',
  'track',
  'scotland'],
 ['side', 'congrat'],
 ['yeaaaah',
  'yippppi',
  'accnt',
  'verifi',
  'rqst',
  'succeed',
  'got',
  'blue',
  'tick',
  'mark',
  'fb',
  'profil',
  'day'],
 ['bhaktisbant',
  'pallaviruhail',
  'one',
  'irresist',
  'flipkartfashionfriday',
  'httptcoebzlvenm'],
 ['dont',
  'like',
  'keep',
  'love',
  'custom',
  'wait',
  'long',
  'hope',
  'enjoy',
  'happi',
  'friday',
  'lwwf',
  'httpstcosmyyriipxi'],
 ['impatientraid',
  'second',
  'thought',
  'there',
  'enough',
  'time',
  'dd',
  'new',
  'short',
  'enter',
  'system',
  'sheep',
  'must',
  'buy'],
 ['jgh', 'go', 'bayan', 'bye'],
 ['act',
  'mischiev',
  'call',
  'etl',


In [ ]:
import numpy as np

labels = np.append(np.ones((len(all_positive_tweets))), np.zeros((len(all_negative_tweets))))

In [ ]:
labels.shape

(10000,)

In [ ]:
def build_freqs(tweeTtok, ys):
    """Build frequencies.
    Input:
        tweeTtok: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    freqs = {}
    for y, tweet in zip(yslist, tweeTtok):
        for word in process_tweet(tweet):
            if isinstance(word, list):
                for w in word:
                    pair = (w, y)
                    if pair in freqs:
                        freqs[pair] += 1
                    else:
                        freqs[pair] = 1
            else:
                pair = (word, y)
                if pair in freqs:
                    freqs[pair] += 1
                else:
                    freqs[pair] = 1
    return freqs


In [ ]:
freqs = build_freqs(tweeTtok, labels)

In [ ]:
freqs

{('followfriday', 1.0): 25,
 ('franceint', 1.0): 1,
 ('pkuchli', 1.0): 1,
 ('milipolpari', 1.0): 1,
 ('top', 1.0): 31,
 ('engag', 1.0): 7,
 ('member', 1.0): 15,
 ('commun', 1.0): 33,
 ('week', 1.0): 84,
 ('lambja', 1.0): 1,
 ('hey', 1.0): 76,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 ('pleas', 1.0): 97,
 ('call', 1.0): 37,
 ('contact', 1.0): 7,
 ('centr', 1.0): 2,
 ('abl', 1.0): 8,
 ('assist', 1.0): 1,
 ('mani', 1.0): 32,
 ('thank', 1.0): 637,
 ('despiteoffici', 1.0): 1,
 ('listen', 1.0): 16,
 ('last', 1.0): 47,
 ('night', 1.0): 69,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 49,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('side', 1.0): 17,
 ('congrat', 1.0): 21,
 ('yeaaaah', 1.0): 1,
 ('yippppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 73,
 ('blue', 1.0): 9,
 ('tick', 1.0): 1,
 ('mark', 1.0): 2,
 ('fb', 1.0): 6,
 ('profil', 1.0): 2,
 ('day', 1.0): 243,
 ('bhaktisbant', 1.0): 17,
 ('pallaviruhail', 1.0): 8,
 ('one', 1.0): 130,
 ('i

In [ ]:
# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 21191


In [ ]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [ ]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [ ]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [ ]:
train_x=process_tweet(train_x)


In [ ]:
train_x

[['followfriday',
  'franceint',
  'pkuchli',
  'milipolpari',
  'top',
  'engag',
  'member',
  'commun',
  'week'],
 ['lambja',
  'hey',
  'jame',
  'odd',
  'pleas',
  'call',
  'contact',
  'centr',
  'abl',
  'assist',
  'mani',
  'thank'],
 ['despiteoffici',
  'listen',
  'last',
  'night',
  'bleed',
  'amaz',
  'track',
  'scotland'],
 ['side', 'congrat'],
 ['yeaaaah',
  'yippppi',
  'accnt',
  'verifi',
  'rqst',
  'succeed',
  'got',
  'blue',
  'tick',
  'mark',
  'fb',
  'profil',
  'day'],
 ['bhaktisbant',
  'pallaviruhail',
  'one',
  'irresist',
  'flipkartfashionfriday',
  'httptcoebzlvenm'],
 ['dont',
  'like',
  'keep',
  'love',
  'custom',
  'wait',
  'long',
  'hope',
  'enjoy',
  'happi',
  'friday',
  'lwwf',
  'httpstcosmyyriipxi'],
 ['impatientraid',
  'second',
  'thought',
  'there',
  'enough',
  'time',
  'dd',
  'new',
  'short',
  'enter',
  'system',
  'sheep',
  'must',
  'buy'],
 ['jgh', 'go', 'bayan', 'bye'],
 ['act',
  'mischiev',
  'call',
  'etl',


In [ ]:
freqs = build_freqs(train_x, train_y)
print(freqs)
# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

{('followfriday', 1.0): 23, ('franceint', 1.0): 1, ('pkuchli', 1.0): 1, ('milipolpari', 1.0): 1, ('top', 1.0): 29, ('engag', 1.0): 7, ('member', 1.0): 13, ('commun', 1.0): 27, ('week', 1.0): 72, ('lambja', 1.0): 1, ('hey', 1.0): 59, ('jame', 1.0): 7, ('odd', 1.0): 2, ('pleas', 1.0): 81, ('call', 1.0): 27, ('contact', 1.0): 4, ('centr', 1.0): 1, ('abl', 1.0): 6, ('assist', 1.0): 1, ('mani', 1.0): 28, ('thank', 1.0): 515, ('despiteoffici', 1.0): 1, ('listen', 1.0): 14, ('last', 1.0): 39, ('night', 1.0): 56, ('bleed', 1.0): 2, ('amaz', 1.0): 40, ('track', 1.0): 5, ('scotland', 1.0): 2, ('side', 1.0): 14, ('congrat', 1.0): 15, ('yeaaaah', 1.0): 1, ('yippppi', 1.0): 1, ('accnt', 1.0): 2, ('verifi', 1.0): 2, ('rqst', 1.0): 1, ('succeed', 1.0): 1, ('got', 1.0): 60, ('blue', 1.0): 8, ('tick', 1.0): 1, ('mark', 1.0): 2, ('fb', 1.0): 4, ('profil', 1.0): 2, ('day', 1.0): 184, ('bhaktisbant', 1.0): 16, ('pallaviruhail', 1.0): 8, ('one', 1.0): 91, ('irresist', 1.0): 2, ('flipkartfashionfriday', 1.0

In [ ]:
# UNQ_C1 GRADED FUNCTION: sigmoid
def sigmoid(z):
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''

    # calculate the sigmoid of z
    h = 1/(1+np.exp(-z))

    return h

In [ ]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]

    for i in range(0, num_iters):

        # get z, the dot product of x and theta
        z = np.dot(x,theta)

        # get the sigmoid of z
        h = sigmoid(z)

        # calculate the cost function
        J = (-1/m)*(np.matmul(np.transpose(y),np.log(h)) + np.matmul(np.transpose(1-y),np.log(1-h)))

        # update the weights theta
        theta = theta - (alpha/m)*np.dot(np.transpose(x),(h-y))

    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [ ]:
def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''

    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3))

    #bias term is set to 1
    x[0,0] = 1

    ### START CODE HERE ###

    # loop through each word in the list of words
    for word in tweet:

        if (word, 1.0) in freqs.keys() :
        # increment the word count for the positive label 1
            x[0,1] += freqs[(word, 1.0)]
        if (word, 0.0) in freqs.keys() :
        # increment the word count for the negative label 0
            x[0,2] += freqs[(word, 0.0)]

    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [ ]:
extract_features(train_x[0], freqs=freqs)

array([[  1., 174.,  60.]])

In [ ]:
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[ 1. 65. 78.]]


In [ ]:
tmp2 = extract_features('hi this is fijaz', freqs)
print(tmp2)

[[ 1.  7. 21.]]


In [ ]:

# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y


In [ ]:
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}." )

The cost after training is 0.69123464.
The resulting vector of weights is [-0.0, 4.949e-05, -2.052e-05].


<ipython-input-83-7c07a5488de2>:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [ ]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###

    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)

    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))

    ### END CODE HERE ###

    return y_pred

In [ ]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.502489
I am bad -> 0.500021
this movie should have been great. -> 0.502135
great -> 0.500159
great great -> 0.500319
great great great -> 0.500478
great great great great -> 0.500637


<ipython-input-86-a6cdc3e04f75>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))


In [ ]:
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

array([[0.50025928]])

In [ ]:
# UNQ_C5 GRADED FUNCTION: test_logistic_regression
def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """

    ### START CODE HERE ###

    # the list for storing predictions
    y_hat = []

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)

        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0.0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator

    accuracy = 0

    for i in range(test_y.shape[0]):
        if test_y[i]==y_hat[i]:
            accuracy+=1

    accuracy=np.float64(accuracy/test_y.shape[0])

    ### END CODE HERE ###

    return accuracy


In [ ]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.5040


In [ ]:
# Feel free to change the tweet below
my_tweet = 'I won!'
def process_tweet(my_tweet):
  return word_tokenize(my_tweet)
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else:
    print('Negative sentiment')

['I', 'won', '!']
[[0.50010083]]
Positive sentiment
